In [2]:
import findspark
findspark.init('/home/realtour/sparkTutorial/spark-2.4.7-bin-hadoop2.7')


In [3]:
#create spark session
import pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
spark = SparkSession \
        .builder \
        .appName('ETL-mysql-to-postgres') \
        .master('local[*]') \
        .config("spark.driver.extraClassPath", "/usr/share/java/mysql-connector-java-8.0.23.jar") \
        .getOrCreate()

spark


In [5]:
orderdetails_df = spark.read.format("jdbc") \
               .option("url","jdbc:mysql://localhost:3306/classicmodels") \
               .option("driver","com.mysql.jdbc.Driver") \
               .option("dbtable","orderdetails") \
               .option("user","root") \
               .option("password","Meta@123").load()


In [10]:
orderdetails_df.limit(4).toPandas()

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2
2,10100,S18_4409,22,75.46,4
3,10100,S24_3969,49,35.29,1


In [11]:
orderdetails_df.printSchema()

root
 |-- orderNumber: integer (nullable = true)
 |-- productCode: string (nullable = true)
 |-- quantityOrdered: integer (nullable = true)
 |-- priceEach: decimal(10,2) (nullable = true)
 |-- orderLineNumber: integer (nullable = true)



In [15]:
#first create table in mysql and then write data
orderdetails_df.write.format('jdbc').options(
    url='jdbc:mysql://localhost:3306/result',
    driver='com.mysql.jdbc.Driver',
    dbtable='orderdetails',
    user='root',
    password='Meta@123').mode('append').save()#we can use 'overwrite' method mode('overwrite')


In [1]:
orderdetails_df.write.format('jdbc').options(
      url='jdbc:postgresql://localhost:5432/result',
      driver='org.postgresql.Driver',
      dbtable='orderdetails',
      user='postgres',
      password='Meta@123').mode('append').save()
